# Editing Features

The ArcGIS API for Python makes programmatic editing of features a breeze. The `edit_features()` method on `FeatureLayer` object can be used for the same. In the example below, we see how to add, update and delete features from a point feature layer

In [ ]:
#connect to GIS
from arcgis.gis import GIS
from IPython.display import display
gis = GIS("portal url", 'username', 'password')

In [ ]:
#search for the feature layer named Ports along west coast
search_result = gis.content.search('title:Ports along west coast')
search_result[0]

In [ ]:
#access the item's feature layers
ports_item = search_result[0]
ports_layers = ports_item.layers
ports_layers

## Query the features
Let us query the features on this layer

In [ ]:
#query all the features and display it on a map
ports_fset = ports_layers[0].query() #an empty query string will return all 
                                        #the features or the first 1000 which ever is smaller

The `sdf` property of `FeatureSet` class is a powerful way to visualize all the features as a pandas dataframe table.

In [ ]:
ports_fset.sdf

## Check if the layer can be edited
To enable a feature layer to be edited, the `edit` capability needs to be turned on. This can be verified by accessing the `capabilities` property on the `FeatureLayer` object:

In [ ]:
ports_flayer = ports_layers[0]
ports_flayer.properties.capabilities

## Editing features

The feature layer does have all editing, updating and deleting capabilities turned on, thus it is ready for edit operations.

### Updating features
From the dataframe above, we notice the short_form field of San Francisco port has a typo. We can update this by calling the `edit_features()` method on the `ports_flayer` `FeatureLayer` object. The `edit_features()` method accepts a `Feature` object with correct parameters set.

In [ ]:
ports_features = ports_fset.features

# select San Francisco feature
sfo_feature = [f for f in ports_features if f.attributes['port_name']=='SAN FRANCISCO'][0]
sfo_feature.attributes

As we can see above, the `short_form` attribute should be changed to `SFO`. Let us edit that and store it as a new `Feature` object

In [ ]:
sfo_edit = sfo_feature
sfo_edit.attributes['short_form'] = 'SFO'

In [ ]:
display(sfo_edit)

Now let us send this feature object to the `edit_features()` method and edit the original feature layer

In [ ]:
update_result = ports_flayer.edit_features(updates=[sfo_edit])
update_result

### Adding features
From the data frame, we also notice that 'Los Angeles', an important port is missing. We can add this to the original layer by calling the `edit_features()` method and passing a dictionary representation of the feature

In [ ]:
# construct a Feature object for Los Angeles.
la_dict = {"attributes": 
           {"latitude": 33.75,
            "longitude": -118.25,
            "country": "US",
            "harborsize": "L",
            "label_position": "SW",
            "port_name": "LOS ANGELES",
            "short_form": "LAX"}, 
           "geometry": 
           {"x": -13044788.958999995, "y": 3857756.351200014}}

In [ ]:
add_result = ports_flayer.edit_features(adds = [la_dict])

In [ ]:
add_result

### Deleting features
From the data frame, we notice 'Redlands', a land locked city is wrongly added as a port. We can remove it by calling the `edit_features()` method and passing the object id of the feature we want to be remove to the `deletes` parameter.

In [ ]:
# find object id for Redlands
Redlands_feature = [f for f in ports_features if f.attributes['port_name'] == 'REDLANDS'][0]
Redlands_objid = Redlands_feature.get_value('objectid')
Redlands_objid

In [ ]:
type(Redlands_objid)

In [ ]:
# pass the object id as a string to the delete parameter
delete_result = ports_flayer.edit_features(deletes=str(Redlands_objid))
delete_result

## Check results of edit operations
We have successfully added, removed and edited features. Let us query the layer once again and display the resutls as a dataframe and verify them.

In [ ]:
ports_fset_edited = ports_flayer.query()
ports_fset_edited.sdf

Thus, we no longer see 'Redlands' which we deleted. We see a new objectid of 23 corresponding to Los Angeles that we added. Also the short_form of San Francisco is corrected to 'SFO'.

The edits we performed on the features are persisted in the feature layer and will reflect in all clients accessing it.